In [8]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os 
import sys
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
from matplotlib.colors import ListedColormap
import folium
from folium.plugins import MarkerCluster
import leafmap

In [2]:
schema = 'projects_research'
table = 'arroyovivo_residuos_hex'
query = f"SELECT * FROM {schema}.{table}"
residuos = aup.gdf_from_query(query, geometry_col='geometry')

print(residuos.shape)
residuos.head(1)

(562, 51)


,hex_id,res,geometry,aluminio,aluminio_quantity,baterias,baterias_quantity,carton,carton_quantity,cenizas,...,unicel,unicel_quantity,vidrio,vidrio_quantity,epi,epi_quantity,total_points,most_common_point,total_quantity,most_common_quantity
0,8a48a20296a7fff,10,"POLYGON ((-100.30269 25.63956, -100.30201 25.6...",13.0,24.0,0.0,0.0,36.0,44.0,0.0,...,207.0,259.0,89.0,137.0,1.0,1.0,1918.0,Plastico,2669.0,Plastico_quantity


In [3]:
residuos.columns

Index(['hex_id', 'res', 'geometry', 'aluminio', 'aluminio_quantity',
       'baterias', 'baterias_quantity', 'carton', 'carton_quantity', 'cenizas',
       'cenizas_quantity', 'ceramica', 'ceramica_quantity', 'electronica',
       'electronica_quantity', 'escombros', 'escombros_quantity', 'goma',
       'goma_quantity', 'llantas', 'llantas_quantity', 'madera',
       'madera_quantity', 'metal', 'metal_quantity', 'organico',
       'organico_quantity', 'otros', 'otros_quantity', 'papel',
       'papel_quantity', 'pesca', 'pesca_quantity', 'plastico',
       'plastico_quantity', 'salud e higiene', 'salud e higiene_quantity',
       'tabaco', 'tabaco_quantity', 'tela', 'tela_quantity', 'unicel',
       'unicel_quantity', 'vidrio', 'vidrio_quantity', 'epi', 'epi_quantity',
       'total_points', 'most_common_point', 'total_quantity',
       'most_common_quantity'],
      dtype='object')

## Dividir Data Sets en Material y Material_quantity

### DataFrame Material

In [7]:
col_material = ['hex_id', 'geometry', 'aluminio', 'baterias', 'carton','cenizas','ceramica','electronica','escombros',
                'goma','llantas','madera','metal','organico','otros','papel','pesca','plastico','salud e higiene',
                'tabaco','tela','unicel','vidrio','epi','total_points', 'most_common_point'] 
data_material = residuos[col_material].copy()
print(data_material.shape)
data_material.head(2)

(562, 26)


,hex_id,geometry,aluminio,baterias,carton,cenizas,ceramica,electronica,escombros,goma,...,pesca,plastico,salud e higiene,tabaco,tela,unicel,vidrio,epi,total_points,most_common_point
0,8a48a20296a7fff,"POLYGON ((-100.30269 25.63956, -100.30201 25.6...",13.0,0.0,36.0,0.0,10.0,1.0,21.0,0.0,...,0.0,1202.0,29.0,13.0,84.0,207.0,89.0,1.0,1918.0,Plastico
1,8a48a202978ffff,"POLYGON ((-100.30331 25.63839, -100.30263 25.6...",10.0,0.0,13.0,2.0,2.0,2.0,25.0,2.0,...,0.0,760.0,12.0,7.0,81.0,57.0,26.0,0.0,1122.0,Plastico


In [9]:
data_material.columns

Index(['hex_id', 'geometry', 'aluminio', 'baterias', 'carton', 'cenizas',
       'ceramica', 'electronica', 'escombros', 'goma', 'llantas', 'madera',
       'metal', 'organico', 'otros', 'papel', 'pesca', 'plastico',
       'salud e higiene', 'tabaco', 'tela', 'unicel', 'vidrio', 'epi',
       'total_points', 'most_common_point'],
      dtype='object')

### DataFrame Material_quantity

In [5]:
col_qty = ['hex_id', 'geometry', 'aluminio_quantity', 'baterias_quantity', 'carton_quantity','cenizas_quantity',
                  'ceramica_quantity','electronica_quantity','escombros_quantity','goma_quantity','llantas_quantity', 
                  'madera_quantity','metal_quantity','organico_quantity','otros_quantity','papel_quantity','pesca_quantity',
                  'plastico_quantity','salud e higiene_quantity','tabaco_quantity','tela_quantity','unicel_quantity',
                  'vidrio_quantity','epi_quantity','total_quantity','most_common_quantity'] 
data_quantity = residuos[col_qty].copy()
print(data_quantity.shape)
data_quantity.head(2)

(562, 26)


,hex_id,geometry,aluminio_quantity,baterias_quantity,carton_quantity,cenizas_quantity,ceramica_quantity,electronica_quantity,escombros_quantity,goma_quantity,...,pesca_quantity,plastico_quantity,salud e higiene_quantity,tabaco_quantity,tela_quantity,unicel_quantity,vidrio_quantity,epi_quantity,total_quantity,most_common_quantity
0,8a48a20296a7fff,"POLYGON ((-100.30269 25.63956, -100.30201 25.6...",24.0,0.0,44.0,0.0,52.0,2.0,39.0,0.0,...,0.0,1479.0,37.0,21.0,108.0,259.0,137.0,1.0,2669.0,Plastico_quantity
1,8a48a202978ffff,"POLYGON ((-100.30331 25.63839, -100.30263 25.6...",33.0,0.0,15.0,2.0,5.0,5.0,59.0,2.0,...,0.0,1640.0,38.0,7.0,170.0,238.0,55.0,0.0,2473.0,Plastico_quantity


In [6]:
data_quantity.columns

Index(['hex_id', 'geometry', 'aluminio_quantity', 'baterias_quantity',
       'carton_quantity', 'cenizas_quantity', 'ceramica_quantity',
       'electronica_quantity', 'escombros_quantity', 'goma_quantity',
       'llantas_quantity', 'madera_quantity', 'metal_quantity',
       'organico_quantity', 'otros_quantity', 'papel_quantity',
       'pesca_quantity', 'plastico_quantity', 'salud e higiene_quantity',
       'tabaco_quantity', 'tela_quantity', 'unicel_quantity',
       'vidrio_quantity', 'epi_quantity', 'total_quantity',
       'most_common_quantity'],
      dtype='object')